<h1>C4 Solution</h1>

<h3>Get the data and copy it to S3</h3>

In [1]:
%%capture
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

In [2]:
%%capture
!unzip dogImages.zip

In [4]:
%%capture
!aws s3 cp dogImages s3://operationalizing-ml-project/ --recursive

<h3>Install and import</h3>

In [5]:
%%capture
!pip install smdebug torch torchvision tqdm

In [28]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
import os

<h3>Set up parameters, estimator, and tuner</h3>

In [7]:
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
}

role = sagemaker.get_execution_role()

objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [8]:
estimator = PyTorch(
    entry_point="hpo.py",
    base_job_name='pytorch_dog_hpo',
    role=role,
    framework_version="1.4.0",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    py_version='py3'
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=2,
    max_parallel_jobs=2,
    objective_type=objective_type
)

<h3>Fit the tuner</h3>

In [9]:
os.environ['SM_CHANNEL_TRAINING']='s3://operationalizing-ml-project/'
os.environ['SM_MODEL_DIR']='s3://operationalizing-ml-project/model/'
os.environ['SM_OUTPUT_DATA_DIR']='s3://operationalizing-ml-project/output/'
tuner.fit({"training": "s3://operationalizing-ml-project/"})

...........................................................................................................................................................................................................................................................................................................................................................!


<h3>Describe the tuning results</h3>


In [11]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
  hyperparameter_tuning_job_name='pytorch-training-220427-0311')

jobs = exp.dataframe()

jobs.sort_values('FinalObjectiveValue', ascending=0)

,batch_size,learning_rate,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,"""64""",0.095946,pytorch-training-220427-0311-002-595b0463,Completed,291.0,2022-04-27 03:12:40+00:00,2022-04-27 03:39:34+00:00,1614.0
1,"""512""",0.019362,pytorch-training-220427-0311-001-9fae4de3,Failed,NaN,2022-04-27 03:12:43+00:00,2022-04-27 03:16:45+00:00,242.0


## Imp: If kernel dies, how to continue from a completed training job

In [12]:
#BetterTrainingJobName='pytorch-training-210623-2156-001-fdd5e081'

In [13]:
#my_estimator = sagemaker.estimator.Estimator.attach(BetterTrainingJobName)


In [14]:
#my_estimator.hyperparameters()

In [15]:
#best_estimator=my_estimator

<h3>Prepare to perform Training on Best Estimator</h3>

In [16]:
best_estimator=tuner.best_estimator()


2022-04-27 03:39:34 Starting - Preparing the instances for training
2022-04-27 03:39:34 Downloading - Downloading input data
2022-04-27 03:39:34 Training - Training image download completed. Training in progress.
2022-04-27 03:39:34 Uploading - Uploading generated training model
2022-04-27 03:39:34 Completed - Training job completed


In [17]:
best_estimator.hyperparameters()

{'_tuning_objective_metric': '"Test Loss"',
 'batch_size': '"64"',
 'learning_rate': '0.09594619140900786',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch_dog_hpo-2022-04-27-03-11-09-615"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-497722403446/pytorch_dog_hpo-2022-04-27-03-11-09-615/source/sourcedir.tar.gz"'}

In [18]:
hyperparameters = {"batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), \
                   "learning_rate": best_estimator.hyperparameters()['learning_rate']}
hyperparameters

{'batch_size': 64, 'learning_rate': '0.09594619140900786'}

In [19]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [20]:
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "1",
        "eval.save_interval": "1"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=1)
)

<h2>Creating an Estimator</h2>

In [30]:
#adjust this cell to accomplish multi-instance training
estimator = PyTorch(
    entry_point='hpo.py',
    base_job_name='dog-pytorch',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.4.0',
    py_version='py3',
    hyperparameters=hyperparameters,
    ## Debugger and Profiler parameters
    rules = rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [31]:
estimator.fit({"training": "s3://operationalizing-ml-project/"}, wait=False)

<h2>Creating an Estimator - Multi-Instance Training,</h2>

In [24]:
###in this cell, create and fit an estimator using multi-instance training
multi_instance_estimator = PyTorch(
    entry_point='hpo.py',
    base_job_name='dog-pytorch',
    role=role,
    instance_count=3,#Setting instance count to 5 for multiinstance training
    instance_type='ml.m5.xlarge', 
    framework_version='1.4.0',
    py_version='py3',
    hyperparameters=hyperparameters,
    ## Debugger and Profiler parameters
    rules = rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [25]:
multi_instance_estimator.fit({"training": "s3://operationalizing-ml-project/"}, wait=False)

<h2>Deployment</h2>

In [34]:
multi_instance_estimator.model_data

KeyError: 'ModelArtifacts'

In [28]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor


In [29]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

In [30]:
pytorch_model = PyTorchModel(model_data=model_location, role=role, entry_point='infernce2.py',py_version='py3',
                             framework_version='1.4',
                             predictor_cls=ImagePredictor)

In [31]:
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.large')


---------------!

In [ ]:
import requests
#request_dict={ "url": "https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg" }
request_dict={ "url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg" }

img_bytes = requests.get(request_dict['url']).content
type(img_bytes)

In [ ]:
from PIL import Image
import io
Image.open(io.BytesIO(img_bytes))

In [ ]:
response=predictor.predict(img_bytes, initial_args={"ContentType": "image/jpeg"})

In [ ]:
import json
response2=predictor.predict(json.dumps(request_dict), initial_args={"ContentType": "application/json"})

In [ ]:
type(response2[0][0])

In [ ]:
response2[0]

In [ ]:
import torch
import numpy as np
np.argmax(response, 1)